In [ ]:
import pandas as pd
df_train_1=pd.read_csv("../data/train_transaction.csv")
df_train_2=pd.read_csv("../data/train_identity.csv")

In [28]:
df_train_1.shape,df_train_2.shape

((590540, 394), (144233, 41))

In [29]:
df_train_1.info(verbose=True,show_counts=True)  ###no missing values in df_train_1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 394 columns):
 #    Column          Non-Null Count   Dtype  
---   ------          --------------   -----  
 0    TransactionID   590540 non-null  int64  
 1    isFraud         590540 non-null  int64  
 2    TransactionDT   590540 non-null  int64  
 3    TransactionAmt  590540 non-null  float64
 4    ProductCD       590540 non-null  object 
 5    card1           590540 non-null  int64  
 6    card2           581607 non-null  float64
 7    card3           588975 non-null  float64
 8    card4           588963 non-null  object 
 9    card5           586281 non-null  float64
 10   card6           588969 non-null  object 
 11   addr1           524834 non-null  float64
 12   addr2           524834 non-null  float64
 13   dist1           238269 non-null  float64
 14   dist2           37627 non-null   float64
 15   P_emaildomain   496084 non-null  object 
 16   R_emaildomain   137291 non-null  obj

In [30]:
df_train_2.info(verbose=True,show_counts=True)  ###no missing values in df_train_2 but the shape is (144233, 41) while df_train_1 shape is (590540, 394) so there are missing values in df_train after merging

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144233 entries, 0 to 144232
Data columns (total 41 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionID  144233 non-null  int64  
 1   id_01          144233 non-null  float64
 2   id_02          140872 non-null  float64
 3   id_03          66324 non-null   float64
 4   id_04          66324 non-null   float64
 5   id_05          136865 non-null  float64
 6   id_06          136865 non-null  float64
 7   id_07          5155 non-null    float64
 8   id_08          5155 non-null    float64
 9   id_09          74926 non-null   float64
 10  id_10          74926 non-null   float64
 11  id_11          140978 non-null  float64
 12  id_12          144233 non-null  object 
 13  id_13          127320 non-null  float64
 14  id_14          80044 non-null   float64
 15  id_15          140985 non-null  object 
 16  id_16          129340 non-null  object 
 17  id_17          139369 non-nul

In [2]:
df_train=pd.merge(df_train_1,df_train_2,how="left",on="TransactionID")
#df_test=pd.merge(df_test_1,df_test_2,how="left",on="TransactionID")

In [3]:
missing_percentage=(df_train.isnull().sum()/len(df_train))*100
missing_percentage=missing_percentage[missing_percentage>0].sort_values(ascending=False)
missing_percentage[missing_percentage>95]

id_24    99.196159
id_25    99.130965
id_07    99.127070
id_08    99.127070
id_21    99.126393
id_26    99.125715
id_23    99.124699
id_22    99.124699
id_27    99.124699
dtype: float64

In [33]:
df_train.dtypes.value_counts()

float64    399
object      31
int64        4
Name: count, dtype: int64

In [4]:
df_train.drop(columns=missing_percentage[missing_percentage>95].index,inplace=True)

#### To get feature importance for time being lets do simpler imputation to handle missing values

In [5]:
category=[]
numerical=[]
for col in df_train.columns:
    if df_train[col].dtype == 'category' or df_train[col].dtype == 'object':
        category.append(col)
    else:
     numerical.append(col)

In [36]:
len(category),len(numerical)

(29, 396)

In [37]:
df_train.shape

(590540, 425)

In [6]:
df_train[category]=df_train[category].astype('object')
df_train[category]=df_train[category].fillna(value='unknown')
df_train[numerical]=df_train[numerical].fillna(df_train[numerical].median())

In [39]:
df_train.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Data columns (total 425 columns):
 #    Column          Non-Null Count   Dtype  
---   ------          --------------   -----  
 0    TransactionID   590540 non-null  int64  
 1    isFraud         590540 non-null  int64  
 2    TransactionDT   590540 non-null  int64  
 3    TransactionAmt  590540 non-null  float64
 4    ProductCD       590540 non-null  object 
 5    card1           590540 non-null  int64  
 6    card2           590540 non-null  float64
 7    card3           590540 non-null  float64
 8    card4           590540 non-null  object 
 9    card5           590540 non-null  float64
 10   card6           590540 non-null  object 
 11   addr1           590540 non-null  float64
 12   addr2           590540 non-null  float64
 13   dist1           590540 non-null  float64
 14   dist2           590540 non-null  float64
 15   P_emaildomain   590540 non-null  object 
 16   R_emaildomain   590540 non-null  obj

In [40]:
df_train["isFraud"].isnull().sum()

np.int64(0)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_train.drop(columns=['isFraud','TransactionID']),df_train['isFraud'],test_size=0.2,random_state=42,stratify=df_train['isFraud'])
for col in category:
    if X_train[col].nunique() <= 100:
        le=LabelEncoder()
        X_train[col]=le.fit_transform(X_train[col])
        X_test[col]=le.transform(X_test[col])
    else:
        frequency=X_train[col].value_counts()/len(X_train)
        X_train[col]=X_train[col].map(frequency)
        X_test[col]=X_test[col].map(frequency)

ValueError: Expected a 2-dimensional container but got <class 'pandas.core.series.Series'> instead. Pass a DataFrame containing a single row (i.e. single sample) or a single column (i.e. single feature) instead.

In [42]:
from lightgbm import LGBMClassifier
model=LGBMClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))


[LightGBM] [Info] Number of positive: 16530, number of negative: 455902
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.207240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 38095
[LightGBM] [Info] Number of data points in the train set: 472432, number of used features: 421
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.034989 -> initscore=-3.317101
[LightGBM] [Info] Start training from score -3.317101
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    113975
           1       0.90      0.43      0.58      4133

    accuracy                           0.98    118108
   macro avg       0.94      0.71      0.78    118108
weighted avg       0.98      0.98      0.97    118108

[[113769    206]
 [  2365   1768]]


In [43]:
importance=model.feature_importances_
features=X_train.columns

In [44]:
importance_table=list(zip(features,importance))
importance_sorted=sorted(importance_table,key=lambda x: x[1],reverse =True)

In [45]:
len(importance_sorted)

423

In [49]:
for name,score in importance_sorted:
 print(name,score)
 if score < 5 :
  df_train.drop(columns=[name],inplace=True,axis=1)

card2 124
card1 123
TransactionAmt 117
TransactionDT 104
addr1 97
C13 78
C1 64
card5 59
R_emaildomain 59
D1 52
D15 52
D4 51
C11 50
P_emaildomain 43
C14 43
dist1 41
D2 41
card3 40
id_02 40
D8 37
C6 35
DeviceInfo 35
ProductCD 34
D10 31
card6 29
M6 29
id_20 29
C2 28
id_30 27
D11 26
C12 25
C5 24
C9 24
M4 24
M5 23
V53 23
D3 21
id_01 21
id_31 21
V285 20
id_33 19
V315 18
C8 17
V45 16
V294 16
V310 16
id_09 16
card4 15
V83 15
V165 15
V38 14
V62 14
V87 14
C3 13
V13 13
V82 13
V281 13
V296 13
V313 13
V317 13
id_14 13
C10 12
V76 12
V283 12
id_19 12
M3 11
V131 11
V139 11
V187 11
id_03 11
id_05 11
D12 10
V56 10
V75 10
V61 9
V99 9
V133 9
V169 9
V307 9
V312 9
V320 9
id_04 9
id_06 9
D5 8
D13 8
V48 8
V54 8
V91 8
id_17 8
dist2 7
V35 7
V55 7
V70 7
V126 7
V171 7
C4 6
V67 6
V73 6
V78 6
V94 6
V128 6
V136 6
V149 6
V156 6
V189 6
V197 6
V220 6
V256 6
V308 6
V309 6
V314 6
D14 5
M2 5
V6 5
V12 5
V19 5
V20 5
V66 5
V96 5
V115 5
V143 5
V206 5
V258 5
V261 5
V262 5
V291 5
V295 5
V311 5
V326 5
id_13 5
id_32 5
D6 4


KeyError: "['D6'] not found in axis"

In [50]:
df_train.shape

(590540, 133)

In [52]:
df_train.to_csv("../data/train_cleaned.csv",index=False)

In [ ]:
import pandas as pd
from mh_sys_gen import MHSysGen
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
df=pd.read_csv("../data/train_cleaned.csv")
df.drop(columns=["TransactionID"],inplace=True)
categorical_feature=[]
for col in df.columns:
    if df[col].dtype=="object":
        categorical_feature.append(col)
nominal_cols=['ProductCD','card4','card6']#low cardinality categorical features
nominal_cols_high=['P_emaildomain','R_emaildomain',"id_30","id_31","DeviceInfo","id_33"]#high cardinality categorical features 
ordinal_cols=['M4','M2','M3','M5','M6']
numeric_cols=[]
for col in df.columns:
    if col not in nominal_cols_high and col not in nominal_cols and col not in ordinal_cols and df[col].dtype!="object":
        if col!="isFraud":
         numeric_cols.append(col)
df=df.sort_values(by="TransactionDT")
split_index=int(0.8*len(df))
train_df = df.iloc[:split_index]
test_df = df.iloc[split_index:]
#frequency encoding 
from sklearn.base import BaseEstimator, TransformerMixin
class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.freq_maps={}
    
    def fit(self,X,y=None):
        X=pd.DataFrame(X)
        for col in X.columns:
            freq_map=X[col].value_counts(normalize=True)
            self.freq_maps[col]=freq_map
        return self
    
    def transform(self,X):
        X=pd.DataFrame(X)
        X_transformed=X.copy()
        for col in X.columns:
            X_transformed[col]=X_transformed[col].map(self.freq_maps[col]).fillna(0)
        return X_transformed
from sklearn.preprocessing import OneHotEncoder ,OrdinalEncoder,StandardScaler
ohe=OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore')
oe=OrdinalEncoder(handle_unknown='use_encoded_value',unknown_value=-1)
scaler=StandardScaler()
df[nominal_cols_high].fillna('missing')
from sklearn.compose import ColumnTransformer
preprocessor=ColumnTransformer(transformers=[
    ('nominal',ohe,nominal_cols),
    ('ordinal',oe,ordinal_cols),
    ('freq',FrequencyEncoder(),nominal_cols_high),
    ('numeric',scaler,numeric_cols)
],remainder='passthrough')
X_train=train_df.drop(columns=['isFraud'])
y_train=train_df['isFraud']
X_test=test_df.drop(columns=['isFraud'])
y_test=test_df['isFraud']
X_train_preprocessed=preprocessor.fit_transform(X_train)
X_test_preprocessed=preprocessor.transform(X_test)
def get_feature_names(preprocessor):
    feature_names=[]
    for name, transformer, columns in preprocessor.transformers_:
        if name != 'remainder':
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
            else:
                names = columns
            feature_names.extend(names)
    return feature_names
x_train_trans=pd.DataFrame(X_train_preprocessed,columns=get_feature_names(preprocessor))
x_test_trans=pd.DataFrame(X_test_preprocessed,columns=get_feature_names(preprocessor))
from sklearn.decomposition import PCA
pca=PCA(n_components=0.90,random_state=42)
x_train_pca=pca.fit_transform(x_train_trans)
x_test_pca=pca.transform(x_test_trans)
pca_cols=[f"[ca_{i}]" for i in range(x_train_pca.shape[1])]
df_aug=pd.DataFrame(x_train_pca, columns=pca_cols)
df_aug['isFraud']=y_train.values
mh_aug=MHSysGen(method="parallel",ratio=15,minority_class=1)
X_aug,y_aug=mh_aug.fit_resample(df_aug,target="isFraud")
from sklearn.ensemble import RandomForestClassifier
model_mhsysgen = RandomForestClassifier(n_estimators=600,
                               max_depth=18,
                               min_samples_split=5,
                               min_samples_leaf=2,
                               max_features="sqrt",
                               class_weight='balanced',
                               random_state=42,
                               )
model_mhsysgen.fit(X_aug,y_aug)
y_pred=model.predict(x_test_pca)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
import numpy as np
from sklearn.neighbors import KDTree

class PWMv5Fast:
    def __init__(self, ratio=20, random_state=42):
        self.ratio = ratio
        np.random.seed(random_state)

    def fit_resample(self, X, y):
        X = np.array(X)
        y = np.array(y)

        classes, counts = np.unique(y, return_counts=True)
        min_class = classes[np.argmin(counts)]
        X_min = X[y == min_class]
        X_maj = X[y != min_class]

        # Precompute covariance noise
        cov = np.cov(X_min.T)
        L = np.linalg.cholesky(cov + np.eye(cov.shape[0])*1e-6)

        # Build KDTree only once
        tree = KDTree(X_maj)

        n_new = len(X_min) * self.ratio

        # 1) Compute majority outward direction ONCE for each minority point
        dists, idxs = tree.query(X_min, k=10)
        maj_centers = np.array([X_maj[idx].mean(axis=0) for idx in idxs])
        outward = X_min - maj_centers
        outward /= np.linalg.norm(outward, axis=1, keepdims=True) + 1e-9

        # 2) Sample synthetic points in a vectorized form
        choices = np.random.randint(0, len(X_min), size=n_new)
        base = X_min[choices]
        out = outward[choices]

        # scaling
        scale = np.random.uniform(1.0, 2.0, size=n_new).reshape(-1, 1)

        # random walk vector, normalized
        rand = np.random.randn(n_new, X.shape[1])
        rand /= np.linalg.norm(rand, axis=1, keepdims=True) + 1e-9

        # covariance noise in vectorized manner
        noise = np.dot(np.random.randn(n_new, X.shape[1]), L.T) * 0.03

        # final synthetic points
        synth = base + out * scale + rand * 0.5 + noise

        X_new = np.vstack([X, synth])
        y_new = np.concatenate([y, np.full(n_new, min_class)])

        return X_new, y_new



p5 = PWMv5Fast(ratio=20)
from sklearn.ensemble import RandomForestClassifier
model_pwm = RandomForestClassifier(n_estimators=600,
                               max_depth=18,
                               min_samples_split=5,
                               min_samples_leaf=2,
                               max_features="sqrt",
                               class_weight='balanced',
                               random_state=42,
                               
                               )
X5, y5 = p5.fit_resample(x_train_pca, y_train)
model_pwm.fit(X5,y5)
y_pred=model_pwm.predict(x_test_pca)
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
